<img src="https://iteso.mx/documents/27014/202031/Logo-ITESO-MinimoH.png"
     align="right"
     width="300"/>

# Modelos SARIMA

## *Modelos no lineales para pronósitico*  - Juan Pablo Colome

---

## **<font color= #0077b6> Objetivos de la Práctica </font>**
1.  Utilizar datos de la API oficial de la MLB.
2.  Analizar la estacionariedad de la serie (Prueba de Dickey-Fuller Aumentada).
3.  Implementar la metodología Box-Jenkins para la selección del orden $(p,d,q)(P,D,Q,s)$.
4.  Utilizar `statsmodels` para realizar un modelo SARIMA.
5.  Generar pronósticos y visualizarlos interactivamente con `plotly`.

In [1]:
# @title Instalación de Librerías y Configuración

!pip install MLB-StatsAPI pmdarima plotly --quiet

import statsapi
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.stattools import adfuller, acf, pacf



## **<font color= #0077b6> Descripción de los datos </font>**

- La API de la MLB consta de múltiples estádisticas y datos de cada partido, usualmente este tipo de información se utiliza para predecir estadísticas de próximos juegos de la temporada.

- Este dataset se enfoca a recopilar la cantidad de runs por partido (sumando ambos equipos) que se juegan en la temporada. La Temporada (Season) es un maratón diario de Abril a Octubre. Cada equipo juega 162 partidos.

- Debido a que suele presentar repeticiones en patrones semanales, se utilizará un modelo SARIMA.

**Recordando:**

La Forma Canónica del modelo multiplicativo $SARIMA(p,d,q)(P,D,Q)_s$ se expresa utilizando el operador de rezago $B$:$$\phi_p(B)\Phi_P(B^s)(1-B)^d(1-B^s)^D Y_t = \theta_q(B)\Theta_Q(B^s)\epsilon_t$$

Donde:

$Y_t$: La serie de tiempo observada

$B$: Operador de rezago $B Y_t = Y_{t-1}

$\phi(B)$ y $\theta(B)$: Polinomios de la parte no estacional (AR y MA).

$\Phi(B^s)$ y $\Theta(B^s)$: Polinomios de la parte estacional (Seasonal AR y Seasonal MA).

$\epsilon_t$: Ruido blanco (Error aleatorio con media 0 y varianza constante $\sigma^2$).

In [2]:
# Función para obtener datos de una temporada completa
games = statsapi.schedule(start_date="2025-03-27", end_date="2025-09-28")
df_games = pd.DataFrame(games)
# Solo juegos de temporada regular terminados
df_clean = df_games[df_games['status'] == 'Final'].copy()

# Total de Carreras en el partido
df_clean['total_runs'] = df_clean['away_score'] + df_clean['home_score']
df_clean['game_date'] = pd.to_datetime(df_clean['game_date'])

# Suma de cada día
ts_mlb = df_clean.groupby('game_date')['total_runs'].sum().asfreq('D').fillna(0)

/var/folders/zq/qf312yqs5wnb29fvj3txfrqm0000gn/T/ipykernel_8488/1825534525.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ts_mlb = df_clean.groupby('game_date')['total_runs'].sum().asfreq('D').fillna(0)


In [3]:
# @title Graficamos la serie de tiempo original
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts_mlb.index, y=ts_mlb.values, mode='lines', name='Carreras Diarias'))

# Resaltar la diferencia entre Lunes/Jueves y Fines de Semana
fig.update_layout(
    title='Volumen Diario de Carreras en la MLB',
    xaxis_title='Fecha',
    yaxis_title='Total Carreras (Runs)'
)
fig.show()

In [4]:
# @title Realizamos pruebas de estacionareidad

def check_stationarity(series, title="Serie Original"):
    result = adfuller(series.dropna())
    print(f'ADF Test: {title}')
    print(f'Estadístico ADF: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    is_stationary = result[1] < 0.05
    print(f"¿Es estacionaria? {'SÍ' if is_stationary else 'NO'}\n")
    return is_stationary

# 1. Revisamos la serie original
check_stationarity(ts_mlb, "Nivel Original")

# 2. Aplicamos Primera Diferencia (d=1)
ts_mlb_diff = ts_mlb.diff()

# 3. Revisamos la serie diferenciada
check_stationarity(ts_mlb_diff, "Primera Diferencia (d=1)")

# Creamos una figura con 2 columnas (Subplots)
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Serie Original (No Estacionaria)", "Serie Diferenciada (Estacionaria d=1)")
)

# Gráfico 1: Serie Original
fig.add_trace(
    go.Scatter(x=ts_mlb.index, y=ts_mlb, name='Original'),
    row=1, col=1
)

# Gráfico 2: Serie Diferenciada
fig.add_trace(
    go.Scatter(x=ts_mlb_diff.index, y=ts_mlb_diff, name='Diferenciada'),
    row=1, col=2
)

# Ajustes de diseño
fig.update_layout(
    title_text="Comparativa: Efecto de la Diferenciación",
    showlegend=False, # Ocultamos leyenda
    height=500
)

fig.show()

ADF Test: Nivel Original
Estadístico ADF: -3.2318
p-value: 0.0182
¿Es estacionaria? SÍ

ADF Test: Primera Diferencia (d=1)
Estadístico ADF: -7.3906
p-value: 0.0000
¿Es estacionaria? SÍ



In [5]:

# @title Graficamos ACF y PACF

ts_analysis = ts_mlb.dropna()

# Parámetros
lags = 30  # 30 días
alpha = 0.05 # Nivel de significancia del 95%

# Cálculo de valores ACF y PACF
acf_vals = acf(ts_analysis, nlags=lags, alpha=alpha)[0][1:]
pacf_vals = pacf(ts_analysis, nlags=lags, alpha=alpha)[0][1:]

# Colocamos manualmente el intervalo de confianza para plotly
n = len(ts_analysis)
conf_interval = 1.96 / np.sqrt(n)


fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Función de Autocorrelación (ACF) - Determina MA(q)",
                                    "Autocorrelación Parcial (PACF) - Determina AR(p)"),
                    vertical_spacing=0.15)

# ACF

fig.add_trace(go.Bar(
    x=list(range(1, lags+1)), y=acf_vals,
    name='ACF', marker_color='rgb(31, 119, 180)', showlegend=False
), row=1, col=1)

# Intervalos de confianza (Sombreado)
fig.add_shape(type="rect",
    x0=0.5, y0=-conf_interval, x1=lags+0.5, y1=conf_interval,
    line=dict(color="rgba(0,0,0,0)"), fillcolor="rgba(0,0,0,0.1)",
    row=1, col=1
)
# Líneas límite
fig.add_hline(y=conf_interval, line_dash="dash", line_color="gray", row=1, col=1)
fig.add_hline(y=-conf_interval, line_dash="dash", line_color="gray", row=1, col=1)

# PACF

fig.add_trace(go.Bar(
    x=list(range(1, lags+1)), y=pacf_vals,
    name='PACF', marker_color='rgb(255, 127, 14)', showlegend=False
), row=2, col=1)

# Intervalos de confianza
fig.add_shape(type="rect",
    x0=0.5, y0=-conf_interval, x1=lags+0.5, y1=conf_interval,
    line=dict(color="rgba(0,0,0,0)"), fillcolor="rgba(0,0,0,0.1)",
    row=2, col=1
)
# Líneas límite
fig.add_hline(y=conf_interval, line_dash="dash", line_color="gray", row=2, col=1)
fig.add_hline(y=-conf_interval, line_dash="dash", line_color="gray", row=2, col=1)


fig.update_layout(
    title='<b>Diagnóstico de Estructura: ACF y PACF</b><br><sup>Serie Diferenciada</sup>',
    template='plotly_white',
    height=700,
    bargap=0.8 # Barras delgadas estilo lollipop
)

# Resaltar lags estacionales (7, 14, 21, 28) con líneas verticales rojas tenues
for i in [7, 14, 21, 28]:
    fig.add_vline(x=i, line_width=1, line_dash="dot", line_color="red", opacity=0.5)

fig.show()

In [6]:
# @title Realizamos modelo y graficamos
TEST_DAYS = 21

train = ts_mlb.iloc[:-TEST_DAYS]
test = ts_mlb.iloc[-TEST_DAYS:]

# Con utilizar una diferenciacion es suficiente, tu decides cual utilizar
# (revisa la forma canónica) para verificar que el modelo despues de simplificar
# ya se trata de una serie estacionaria
model = SARIMAX(train,
                order=(5, 0, 5),
                seasonal_order=(2, 0, 2, 7))

results = model.fit(disp=False)

# Predecimos n pasos hacia el futuro (donde n = tamaño del test)
forecast_object = results.get_forecast(steps=len(test))
forecast_vals = forecast_object.predicted_mean
conf_int = forecast_object.conf_int(alpha=0.05) # Intervalo del 95%

# Metricas de error
rmse = np.sqrt(mean_squared_error(test, forecast_vals))
# Ojo con el MAPE, hay algunos días con 0 carreras y esto da valores muy grandes
mape = mean_absolute_percentage_error(test, forecast_vals)
mae = mean_absolute_error(test, forecast_vals)


print(f"\n--- Errores del modelo ---")
print(f"RMSE: {rmse:.2f} Carreras")
print(f"MAPE: {mape:.2%}")
print(f"MAE: {mae:.2f} Carreras")

print(results.summary())

# Grafica
fig = go.Figure()

# Train
fig.add_trace(go.Scatter(
    x=train.index, y=train,
    mode='lines',
    name='Train',
    line=dict(color='rgba(100, 100, 100, 0.6)', width=1.5)
))

# Test
fig.add_trace(go.Scatter(
    x=test.index, y=test,
    name='Test',
    line=dict(color='#1f77b4', width=3),
    marker=dict(size=6)
))

# Forecast
fig.add_trace(go.Scatter(
    x=test.index, y=forecast_vals,
    name='SARIMA',
    line=dict(color='#ff7f0e', width=3, dash='dot')
))

# Intervalos de Confianza
fig.add_trace(go.Scatter(
    x=conf_int.index, y=conf_int.iloc[:, 0],
    mode='lines', line=dict(width=0), showlegend=False, hoverinfo='skip'
))
fig.add_trace(go.Scatter(
    x=conf_int.index, y=conf_int.iloc[:, 1],
    mode='lines', line=dict(width=0), fill='tonexty',
    fillcolor='rgba(255, 127, 14, 0.2)',
    name='Int. Confianza 95%', hoverinfo='skip'
))

# Titulos
fig.update_layout(
    title=f'<b>Modelo SARIMA: Pronóstico de Carreras MLB</b><br><sup>',
    xaxis_title='Fecha',
    yaxis_title='Total de Carreras',
    legend=dict(x=0, y=1, bgcolor='rgba(255,255,255,0.8)'),
    hovermode="x unified"
)

fig.show()

/Users/carlosmoreno/.pyenv/versions/3.10.12/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

/Users/carlosmoreno/.pyenv/versions/3.10.12/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:997: UserWarning:

Non-stationary starting seasonal autoregressive Using zeros as starting parameters.




--- Errores del modelo ---
RMSE: 25.43 Carreras
MAPE: 25.16%
MAE: 19.74 Carreras
                                       SARIMAX Results                                        
Dep. Variable:                             total_runs   No. Observations:                  165
Model:             SARIMAX(5, 0, 5)x(2, 0, [1, 2], 7)   Log Likelihood                -789.376
Date:                                Sun, 22 Feb 2026   AIC                           1608.751
Time:                                        22:20:43   BIC                           1655.341
Sample:                                    03-27-2025   HQIC                          1627.664
                                         - 09-07-2025                                         
Covariance Type:                                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------


/Users/carlosmoreno/.pyenv/versions/3.10.12/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

